This notebook will automatically scrape and update the database day after day.

In [1]:
#Import libraries
import urllib
from bs4 import BeautifulSoup
from selenium  import webdriver
import csv
import datetime
import re
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import numpy as np

#Create or load database
# Define a database name
# Set your postgres username
dbname = 'indeed_posts_v5'
username = 'christopherluciuk' # change this to your username
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

#Get soup from landing page for a starting index
def get_soup(field,loc,start):
    driver = webdriver.Chrome('/Users/christopherluciuk/Downloads/chromedriver')
    #loc is the location
    #field is the search term
    #The last concatenation lets us navigate through pages
    driver.get('https://www.indeed.com/jobs?q='+field+'&l='+loc+'&start='+str(start))
    soup = BeautifulSoup(driver.page_source,"lxml")
    driver.close()
    return soup
#Get soup from a specific job post given the url
def get_soup_post(url):
    driver = webdriver.Chrome('/Users/christopherluciuk/Downloads/chromedriver')
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,"lxml")
    driver.close()
    return soup
#Get the job post links from the landing page
def get_links(soup):
    for item in soup.find_all('a', href=True):
        m = re.search('/pagead/',str(item['href'])) #sponsored ads
        n = re.search('/rc/',str(item['href'])) #regular content
        o = re.search('/clk?',str(item['href']))
        if m is None:
            pass
        else:
            links.append('https://www.indeed.com'+str(item['href']))
        if n is None:
            pass
        else:
            links.append('https://www.indeed.com'+str(item['href']))
    return links
#Get job info from a specific url (after getting soup)
def get_jobsum(soup):
    for item in soup.find_all(name='span', attrs={'class':'summary'}):
        if item is None:
            summary = ''
        else:
            summary = str(item.text.encode('utf-8').strip())
    return summary
#Get metadata from the landing page
def get_metadata(soup,field,loc):
    data = []
    now = datetime.datetime.now()
    date = now.strftime("%Y-%m-%d")
    for item in soup.find_all(name='div', attrs={'class':'row result clickcard'}):
        if item.find(name='span',attrs={'class':'company'}) is None:
            company_name = ''
        else:
            company_name = str(item.find(name='span',attrs={'class':'company'}).text.encode('utf-8').strip())
        if item.find(name='a', attrs={'data-tn-element':'jobTitle'}) is None:
            job_title= ''
        else:
            job_title = str(item.find(name='a', attrs={'data-tn-element':'jobTitle'})['title'])
        if item.find(name='span',attrs={'class':'date'}) is None:
            age = 'None'
        else:    
            age = str(item.find(name='span',attrs={'class':'date'}).text.encode('utf-8').strip())
        
        n = re.search('/rc/',str(item.find(name='a',attrs={'data-tn-element':'jobTitle'})['href']))
        if n is None:
            url = ''
        else:
            url_snip = item.find(name='a',attrs={'data-tn-element':'jobTitle'})['href']
            url_split = url_snip.split('?')
            url = 'https://www.indeed.com/viewjob?'+url_split[1]
        
        entry = [job_title, company_name, age, date, field, loc, url]
        data.append(entry)
    return data
#Get job summaries from list of urls/metadata
def get_summaries(data):
    for i in range(len(data)):
        URL_INDEX = 6 #Hardcoded in
        url = data[i][URL_INDEX] 
        if url == '':
            summary = ''
        else:
            #Open this page and scrape
            soup = get_soup_post(url)
            summary = get_jobsum(soup)
        data[i].append(summary)
#Get the total number of posts
def get_numposts(field,loc):
    driver = webdriver.Chrome('/Users/christopherluciuk/Downloads/chromedriver')
    driver.get('https://www.indeed.com/jobs?q='+field+'&l='+loc+'&start=0')
    soup = BeautifulSoup(driver.page_source,"lxml")
    driver.close()
    tot_num = str(soup.find(name='div', attrs={'id':'searchCount'}).text.encode('utf-8').strip())
    return int(tot_num.split()[3].replace(',',''))

postgres://christopherluciuk@localhost/indeed_posts_v5
True


Here's the actual scraping algorithm. Define keyword and location AND have condition for picking the number of posts.

In [7]:
#Define the location and keyword and a list of starting indices for pagination
loc = 'new+york'
field = [
    'maintenance+supervisor',
    'department+supervisor',
    'assembly+supervisor',
    'shop+supervisor',
    'plant+supervisor',
    'sales+representative',
    'service+representative',
    'customer+service',
    'human+resources',
    'room+attendant',
    'machine+operator',
    'operator+helper',
    'press+operator',
    'machine+tender',
    'saw+operator',
    'information+systems',
    'nurse+practitioner',
    'information+technology',
    'design+engineer',
    'research+scientist',
    'teacher',
    'faculty+member',
    'special+education',
    'associate+professor',
    'social+worker',
    'service+technician',
    'plant+operator',
    'truck+driver',
    'repair+technician',
    'mill+operator'
        ]

job_key = 1 #Need to iterate through this eventually... Most recent Jan20 scrape
print(field[job_key])
start = 0
end1 = get_numposts(field[job_key],loc)
end2 = 200
end = min(end1,end2)
num_per_page = 10
indices = range(start,end,num_per_page)
data = []
for i in indices:
    soup = get_soup(field[job_key], loc, i)
    data.extend(get_metadata(soup,field[job_key],loc))
    
#Get summaries of posts
get_summaries(data)

#Dump into database and check for unique id
#Make connection
con = None
con = psycopg2.connect(database = dbname, user = username)
cur = con.cursor()

#How to add python array to database
sql_query = '''SELECT * FROM data_table ORDER BY ID DESC LIMIT 1;'''
data_out = pd.read_sql_query(sql_query,con)
last_val = int(data_out['id'].values)

for i in range(len(data)):
    item_index = i + (last_val+1) #Need to add this criteria when the database exists!
    data[i].insert(0,item_index)
    cur.execute("INSERT INTO data_table VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s)", (data[i]))
    
con.commit()
cur.close()
con.close()

mill+operator


Sanity checks...

In [8]:
con = None
con = psycopg2.connect(database = dbname, user = username)
cur = con.cursor()
# query:
sql_query = """
SELECT * FROM data_table;
"""
data_out = pd.read_sql_query(sql_query,con)

cur.close()
con.close()
data_out

,id,title,company,date,date_scraped,keyword,location,url,summary
0,0,Maintenance Supervisor,b'QBS',None,2018-01-15,maintenance+supervisor,new+york,,
1,1,Repair and Maintenance Manager,b'MOVE Systems International LLC',None,2018-01-15,maintenance+supervisor,new+york,,
2,2,Maintenance Supervisor,b'Workforce One BX',b'13 days ago',2018-01-15,maintenance+supervisor,new+york,,
3,3,Facilities Manager,b'Cornell University',b'30+ days ago',2018-01-15,maintenance+supervisor,new+york,https://www.indeed.com/viewjob?jk=ce7f7237f778...,"b""Responsible for the repair, maintenance and ..."
4,4,Facilities Manager,b'Burgess Group',b'30+ days ago',2018-01-15,maintenance+supervisor,new+york,https://www.indeed.com/viewjob?jk=73a1b3bc90f0...,b'Proposed Job Title: Manager - Facilities\n\n...
5,5,Facilities Assistant,b'Rensselaer Polytechnic Institute',b'7 days ago',2018-01-15,maintenance+supervisor,new+york,https://www.indeed.com/viewjob?jk=fadb8beebdcd...,b'The incumbent supports Rensselaer Union even...
6,6,One Team Facilities Supervisor,b'Deloitte',b'10 days ago',2018-01-15,maintenance+supervisor,new+york,https://www.indeed.com/viewjob?jk=05ea6d0db881...,"b""Deloitte Services LP includes internal suppo..."
7,7,Maintenance Supervisor,b'Careers - Win',b'10 hours ago',2018-01-15,maintenance+supervisor,new+york,https://www.indeed.com/viewjob?jk=f1126052f8bf...,b'The Maintenance Supervisor in the absence of...
8,8,Facilities Supervisor,b'Madame Tussauds New York',b'30+ days ago',2018-01-15,maintenance+supervisor,new+york,https://www.indeed.com/viewjob?jk=486679000942...,"b""The Maintenance Manager is a vital role to T..."
9,9,Manager Maintenance - JFK,"b'Atlas Air, Inc.'",b'9 days ago',2018-01-15,maintenance+supervisor,new+york,https://www.indeed.com/viewjob?jk=67f801038439...,b'Overview\nThis position is responsible for m...
